<a href="https://colab.research.google.com/github/EDafee1/ir-learn/blob/main/Final_Project_Stg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2><strong>Colab Settings

In [1]:
!pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 18.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached Cython-0.29.36-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 8.6 MB/s eta 0:00:00
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp310-cp310-linux_x86_64.whl size=3039185 sha256=581935a05a3b07519aa25f956ca9ed94faefb19880c9fb6331b6ff508d2f3ebe
  Stored in dir

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [4]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

<h2><strong>Data Preprocessing

In [5]:
path = '/content/gdrive/MyDrive/UDINUS/Information Retrieval/Amazon_Unlocked_Mobile.csv'

In [117]:
df = pd.read_csv(path)
# df = df[0:8000]
df

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0
...,...,...,...,...,...,...
413835,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,5,another great deal great price,0.0
413836,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,3,Ok,0.0
413837,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,5,Passes every drop test onto porcelain tile!,0.0
413838,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,3,I returned it because it did not meet my needs...,0.0


In [118]:
df = df[['Brand Name', 'Reviews']].astype(str)
df.head()

,Brand Name,Reviews
0,Samsung,I feel so LUCKY to have found this used (phone...
1,Samsung,"nice phone, nice up grade from my pantach revu..."
2,Samsung,Very pleased
3,Samsung,It works good but it goes slow sometimes but i...
4,Samsung,Great phone to replace my lost phone. The only...


In [119]:
df = df[(df['Brand Name'] == 'Samsung') | (df['Brand Name'] == 'Apple') | (df['Brand Name'] == 'Nokia')]

In [120]:
df['Brand Name'].unique()

array(['Samsung', 'Nokia', 'Apple'], dtype=object)

In [121]:
df['Brand Name'].value_counts()

Samsung    65747
Apple      58186
Nokia      16806
Name: Brand Name, dtype: int64

In [122]:
df_samsung = df[df['Brand Name'] == 'Samsung']
df_samsung = df_samsung[0:5000]
df_apple = df[df['Brand Name'] == 'Apple']
df_apple = df_apple[0:5000]
df_nokia = df[df['Brand Name'] == 'Nokia']
df_nokia = df_nokia[0:5000]

df_sam_app = df_samsung.append(df_apple, ignore_index=True)
df = df_sam_app.append(df_nokia, ignore_index=True)

In [123]:
def preprocess_text(text):
    tokens = word_tokenize(text.lower())

    filtered_tokens = [token for token in tokens if token not in stopwords.words('english')]

    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]

    processed_text = ' '.join(lemmatized_tokens)
    return processed_text

In [124]:
df['Reviews'] = df['Reviews'].apply(preprocess_text)
df

,Brand Name,Reviews
0,Samsung,feel lucky found used ( phone u & used hard ) ...
1,Samsung,"nice phone , nice grade pantach revue . clean ..."
2,Samsung,pleased
3,Samsung,work good go slow sometimes good phone love
4,Samsung,great phone replace lost phone . thing volume ...
...,...,...
14995,Nokia,got item described website . great service . p...
14996,Nokia,"siempre preferido los telefonos nokia , tengo ..."
14997,Nokia,phone nice feature display problem . phone fel...
14998,Nokia,great phone ... exceeded description saw onlin...


<h2><strong>Sentiment Analysis

In [125]:
analyzer = SentimentIntensityAnalyzer()

In [126]:
def get_sentiment(text):
    scores = analyzer.polarity_scores(text)
    sentiment = 1 if scores['pos'] > 0 else 0
    return sentiment

In [127]:
df['sentiment'] = df['Reviews'].apply(get_sentiment)
df.head()

,Brand Name,Reviews,sentiment
0,Samsung,feel lucky found used ( phone u & used hard ) ...,1
1,Samsung,"nice phone , nice grade pantach revue . clean ...",1
2,Samsung,pleased,1
3,Samsung,work good go slow sometimes good phone love,1
4,Samsung,great phone replace lost phone . thing volume ...,1


In [128]:
df_pos = df[df['sentiment'] == 1]
df_neg = df[df['sentiment'] == 0]

In [101]:
# df_pos_samsung = df_pos[df_pos['Brand Name'] == 'Samsung']
# df_pos_apple = df_pos[df_pos['Brand Name'] == 'Apple']
# df_pos_nokia = df_pos[df_pos['Brand Name'] == 'Nokia']

# df_neg_samsung = df_neg[df_neg['Brand Name'] == 'Samsung']
# df_neg_apple = df_neg[df_neg['Brand Name'] == 'Apple']
# df_neg_nokia = df_neg[df_neg['Brand Name'] == 'Nokia']

<h2><strong>Topics Modelling

In [129]:
import pandas as pd
import numpy as np
from bertopic import BERTopic

In [193]:
docs_pos = df_pos.Reviews.to_list()
docs_neg = df_neg.Reviews.to_list()

In [106]:
# docs_pos_samsung = df_pos_samsung.Reviews.to_list()
# docs_pos_apple = df_pos_apple.Reviews.to_list()
# docs_pos_nokia = df_pos_nokia.Reviews.to_list()

# docs_neg_samsung = df_neg_samsung.Reviews.to_list()
# docs_neg_apple = df_neg_apple.Reviews.to_list()
# docs_neg_nokia = df_neg_nokia.Reviews.to_list()

In [194]:
model_pos = BERTopic(verbose=True)
model_neg = BERTopic(verbose=True)

In [107]:
# model_pos_samsung = BERTopic(verbose=True)
# model_pos_apple = BERTopic(verbose=True)
# model_pos_nokia = BERTopic(verbose=True)

# model_neg_samsung = BERTopic(verbose=True)
# model_neg_apple = BERTopic(verbose=True)
# model_neg_nokia = BERTopic(verbose=True)

In [108]:
# # POS SAMSUNG
# topics, probs = model_pos_samsung.fit_transform(docs_pos_samsung)

# res_pos_samsung = model_pos_samsung.get_topic_info()
# res_pos_samsung.head()

2023-12-08 14:34:21,566 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/131 [00:00<?, ?it/s]

2023-12-08 14:35:49,924 - BERTopic - Embedding - Completed ✓
2023-12-08 14:35:49,926 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2023-12-08 14:35:57,076 - BERTopic - Dimensionality - Completed ✓
2023-12-08 14:35:57,081 - BERTopic - Cluster - Start clustering the reduced embeddings
2023-12-08 14:35:57,208 - BERTopic - Cluster - Completed ✓
2023-12-08 14:35:57,215 - BERTopic - Representation - Extracting topics from clusters using representation models.
2023-12-08 14:35:57,419 - BERTopic - Representation - Completed ✓


In [111]:
# # POS APPLE
# topics, probs = model_pos_apple.fit_transform(docs_pos_apple)

# res_pos_apple = model_pos_apple.get_topic_info()
# res_pos_apple.head()

2023-12-08 14:37:26,833 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/125 [00:00<?, ?it/s]

2023-12-08 14:38:21,873 - BERTopic - Embedding - Completed ✓
2023-12-08 14:38:21,881 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2023-12-08 14:38:44,145 - BERTopic - Dimensionality - Completed ✓
2023-12-08 14:38:44,152 - BERTopic - Cluster - Start clustering the reduced embeddings
2023-12-08 14:38:44,319 - BERTopic - Cluster - Completed ✓
2023-12-08 14:38:44,329 - BERTopic - Representation - Extracting topics from clusters using representation models.
2023-12-08 14:38:44,600 - BERTopic - Representation - Completed ✓


,Topic,Count,Name,Representation,Representative_Docs
0,-1,683,-1_iphone_make_nt_button,"[iphone, make, nt, button, unlocked, sure, pho...","[purchased phone reading thoroughly , another ..."
1,0,148,0_scratch_scratched_nothing_heavy,"[scratch, scratched, nothing, heavy, screen, t...","[phone pretty heavy scratch front back , plus ..."
2,1,102,1_good___,"[good, , , , , , , , , ]","[good ...., good, good]"
3,2,66,2_3gs_3g_4g_solid,"[3gs, 3g, 4g, solid, 2g, didnt, model, extreml...","[good iphone 3gs ., what't difference iphone 3..."
4,3,65,3_contract_probs_cool_index,"[contract, probs, cool, index, heat, collectio...","[great phone , contract phone &, nice color co..."


In [112]:
# # POS NOKIA
# topics, probs = model_pos_nokia.fit_transform(docs_pos_nokia)

# res_pos_nokia = model_pos_nokia.get_topic_info()
# res_pos_nokia.head()

2023-12-08 14:39:13,283 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/125 [00:00<?, ?it/s]

2023-12-08 14:41:15,972 - BERTopic - Embedding - Completed ✓
2023-12-08 14:41:15,974 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2023-12-08 14:41:34,400 - BERTopic - Dimensionality - Completed ✓
2023-12-08 14:41:34,408 - BERTopic - Cluster - Start clustering the reduced embeddings
2023-12-08 14:41:34,585 - BERTopic - Cluster - Completed ✓
2023-12-08 14:41:34,593 - BERTopic - Representation - Extracting topics from clusters using representation models.
2023-12-08 14:41:34,986 - BERTopic - Representation - Completed ✓


,Topic,Count,Name,Representation,Representative_Docs
0,-1,1182,-1_phone_nt_nokia_good,"[phone, nt, nokia, good, work, one, call, use,...",[one exact phone 2 year 'm happy . wife wanted...
1,0,193,0_product_excellent_item_condition,"[product, excellent, item, condition, recommen...","[excellent price excellent product ., excellen..."
2,1,153,1_basic_simple_easy_use,"[basic, simple, easy, use, great, little, call...",[really amazing simple 3.5g phone . 's great g...
3,2,147,2_screen_nt_like_phone,"[screen, nt, like, phone, apps, battery, iphon...","[phone amazing . $ 120 , nokia really hit nail..."
4,3,85,3_price_excellent_good_easy,"[price, excellent, good, easy, movistar, term,...","[great phone ! practical affordable , easy use..."


In [115]:
# res_pos_samsung['Brand Name'] = 'Samsung'
# res_pos_apple['Brand Name'] = 'Apple'
# res_pos_nokia['Brand Name'] = 'Nokia'

# res_pos_sam_app = res_pos_samsung.append(res_pos_apple, ignore_index=True)
# res_pos = res_pos_sam_app.append(res_pos_nokia, ignore_index=True)

In [133]:
topics, probs = model_pos.fit_transform(docs_pos)

# res_pos = model_pos.get_topic_info()

2023-12-08 14:54:53,406 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/380 [00:00<?, ?it/s]

2023-12-08 14:59:04,585 - BERTopic - Embedding - Completed ✓
2023-12-08 14:59:04,586 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2023-12-08 14:59:12,962 - BERTopic - Dimensionality - Completed ✓
2023-12-08 14:59:12,965 - BERTopic - Cluster - Start clustering the reduced embeddings
2023-12-08 14:59:13,396 - BERTopic - Cluster - Completed ✓
2023-12-08 14:59:13,403 - BERTopic - Representation - Extracting topics from clusters using representation models.
2023-12-08 14:59:14,082 - BERTopic - Representation - Completed ✓


In [195]:
topics, probs = model_neg.fit_transform(docs_neg)

2023-12-08 16:15:53,865 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/89 [00:00<?, ?it/s]

2023-12-08 16:16:22,181 - BERTopic - Embedding - Completed ✓
2023-12-08 16:16:22,186 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2023-12-08 16:16:32,689 - BERTopic - Dimensionality - Completed ✓
2023-12-08 16:16:32,696 - BERTopic - Cluster - Start clustering the reduced embeddings
2023-12-08 16:16:32,811 - BERTopic - Cluster - Completed ✓
2023-12-08 16:16:32,822 - BERTopic - Representation - Extracting topics from clusters using representation models.
2023-12-08 16:16:32,975 - BERTopic - Representation - Completed ✓


In [149]:
topics_res_pos = model_pos.get_topic_info()
topics_res_pos.head()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,4150,-1_phone_call_use_nokia,"[phone, call, use, nokia, nt, one, battery, sc...",[say phone excellent value every aspect . feat...
1,0,293,0_good_fone_oh_clean,"[good, fone, oh, clean, verry, pone, exelent, ...","[good, good, good]"
2,1,161,1_scratch_condition_scratched_dent,"[scratch, condition, scratched, dent, glass, c...","[great far . issue . product scratch !, good c..."
3,2,142,2_excellent_ver_ad_en,"[excellent, ver, ad, en, slow, great, new, goo...","[excellent, excellent, excellent]"
4,3,142,3_product_excellent_seller_price,"[product, excellent, seller, price, recommend,...","[excellent quality product nice , good seller,..."


In [196]:
topics_res_neg = model_neg.get_topic_info()
topics_res_neg.head()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,493,-1_phone_nt_month_one,"[phone, nt, month, one, charge, hear, screen, ...","[wanted cell phone qwerty keyboard , 's got . ..."
1,0,399,0_unlocked_sim_card_locked,"[unlocked, sim, card, locked, unlock, gsm, use...","[phone n't unlocked, work sim card, work sim c..."
2,1,158,1_de_la_el_que,"[de, la, el, que, lo, en, muy, un, este, se]",[compré este teléfono para mi esposa por el pr...
3,2,140,2_excelente_xcelente_expectativas_cumpli,"[excelente, xcelente, expectativas, cumpli, fa...","[excelente, excelente, excelente]"
4,3,57,3_returned_defective_back_item,"[returned, defective, back, item, sent, refund...","[returned, work properly , returned full refun..."


In [147]:
res_pos = pd.DataFrame({"Document": docs_pos, "Topic": topics})
res_pos.head()

,Document,Topic
0,feel lucky found used ( phone u & used hard ) ...,22
1,"nice phone , nice grade pantach revue . clean ...",-1
2,pleased,144
3,work good go slow sometimes good phone love,79
4,great phone replace lost phone . thing volume ...,4


In [197]:
res_neg = pd.DataFrame({"Document": docs_neg, "Topic": topics})
res_neg.head()

,Document,Topic
0,battery old & used barely hold charge . otherw...,4
1,"described , fast ship !",14
2,battery drain complete charge 0 30 minute .,4
3,exceeded expectation,21
4,phone randomly powered wo n't turn,33


In [157]:
res_pos['Result'] = 'X'
for a in range(len(res_pos)):
  for b in range(len(topics_res_pos)):
    if res_pos['Topic'][a] == topics_res_pos['Topic'][b]:
      res_pos['Result'][a] = topics_res_pos['Name'][b]

In [198]:
res_neg['Result'] = 'X'
for a in range(len(res_neg)):
  for b in range(len(topics_res_neg)):
    if res_neg['Topic'][a] == topics_res_neg['Topic'][b]:
      res_neg['Result'][a] = topics_res_neg['Name'][b]

In [158]:
res_pos

,Document,Topic,Result
0,feel lucky found used ( phone u & used hard ) ...,22,22_stoled_ihad_new_happy
1,"nice phone , nice grade pantach revue . clean ...",-1,-1_phone_call_use_nokia
2,pleased,144,144_happy_enjoy_thankful_im
3,work good go slow sometimes good phone love,79,79_ty_tryna_voxox_aint
4,great phone replace lost phone . thing volume ...,4,4_button_home_stick_working
...,...,...,...
12155,'ve using cell phone week satisfied phone . ob...,-1,-1_phone_call_use_nokia
12156,got item described website . great service . p...,-1,-1_phone_call_use_nokia
12157,phone nice feature display problem . phone fel...,-1,-1_phone_call_use_nokia
12158,great phone ... exceeded description saw onlin...,-1,-1_phone_call_use_nokia


In [199]:
res_neg

,Document,Topic,Result
0,battery old & used barely hold charge . otherw...,4,4_battery_charge_hour_minute
1,"described , fast ship !",14,14_fast_described_shipped_quickly
2,battery drain complete charge 0 30 minute .,4,4_battery_charge_hour_minute
3,exceeded expectation,21,21_expected_expectation_met_lent
4,phone randomly powered wo n't turn,33,33_shuts_wo_turn_defective
...,...,...,...
2835,cuando compre este movil gusto pero despues de...,1,1_de_la_el_que
2836,"excelente , e la unica palabra que le queda es...",1,1_de_la_el_que
2837,"el telefono e excelente , e facil de manejar ,...",1,1_de_la_el_que
2838,funciona perfectamente en costa rica.no borrar...,1,1_de_la_el_que


In [176]:
# res_pos['Document'] = res_pos['Document'].astype(str)
# df_pos['Reviews'] = df_pos['Reviews'].astype(str)
df_pos.reset_index(inplace = True)
df_pos['Reviews'][20]

'pros-beautiful screen , capable running chrome , take good picture , keyboard comfy , fit pocket fit hand , play really good video gamescons-no bluetooth , came late , run fast really quick , doesnt play 1080p videos.but would still recommend like talk text lot like watch lot movie'

In [200]:
df_neg.reset_index(inplace = True)

In [177]:
res_pos['Brand'] = 'X'
for a in range(len(res_pos)):
  for b in range(len(df_pos)):
    if res_pos['Document'][a] == df_pos['Reviews'][b]:
      res_pos['Brand'][a] = df_pos['Brand Name'][b]

In [201]:
res_neg['Brand'] = 'X'
for a in range(len(res_neg)):
  for b in range(len(df_neg)):
    if res_neg['Document'][a] == df_neg['Reviews'][b]:
      res_neg['Brand'][a] = df_neg['Brand Name'][b]

In [189]:
res_pos['Result'] = res_pos['Result'].str.replace('\d+', '')
res_pos['Result'] = res_pos['Result'].replace('_', ' ', regex=True)
res_pos['Result'] = res_pos['Result'].replace('-', '', regex=True)
res_pos

,Document,Topic,Result,Brand
0,feel lucky found used ( phone u & used hard ) ...,22,stoled ihad new happy,Samsung
1,"nice phone , nice grade pantach revue . clean ...",-1,phone call use nokia,Samsung
2,pleased,144,happy enjoy thankful im,Samsung
3,work good go slow sometimes good phone love,79,ty tryna voxox aint,Samsung
4,great phone replace lost phone . thing volume ...,4,button home stick working,Samsung
...,...,...,...,...
12155,'ve using cell phone week satisfied phone . ob...,-1,phone call use nokia,Nokia
12156,got item described website . great service . p...,-1,phone call use nokia,Nokia
12157,phone nice feature display problem . phone fel...,-1,phone call use nokia,Nokia
12158,great phone ... exceeded description saw onlin...,-1,phone call use nokia,Nokia


In [202]:
res_neg['Result'] = res_neg['Result'].str.replace('\d+', '')
res_neg['Result'] = res_neg['Result'].replace('_', ' ', regex=True)
res_neg['Result'] = res_neg['Result'].replace('-', '', regex=True)
res_neg

,Document,Topic,Result,Brand
0,battery old & used barely hold charge . otherw...,4,battery charge hour minute,Samsung
1,"described , fast ship !",14,fast described shipped quickly,Samsung
2,battery drain complete charge 0 30 minute .,4,battery charge hour minute,Samsung
3,exceeded expectation,21,expected expectation met lent,Samsung
4,phone randomly powered wo n't turn,33,shuts wo turn defective,Samsung
...,...,...,...,...
2835,cuando compre este movil gusto pero despues de...,1,de la el que,Nokia
2836,"excelente , e la unica palabra que le queda es...",1,de la el que,Nokia
2837,"el telefono e excelente , e facil de manejar ,...",1,de la el que,Nokia
2838,funciona perfectamente en costa rica.no borrar...,1,de la el que,Nokia


In [190]:
res_pos_raw = res_pos[['Brand', 'Result']]
res_pos_new = res_pos_raw.groupby(['Brand', 'Result']).first()
res_pos_new

Empty DataFrame
Columns: []
Index: [(Apple,   itis itworked defientely), (Apple,  activate esn linked balance), (Apple,  activating verizon cellmania activate), (Apple,  advertized strongly stock forward), (Apple,  africa ghana region advertisement), (Apple,  amazingit iphoneand tmoible haha), (Apple,  aplications month  ), (Apple,  apple told amazon girlfriend), (Apple,  apps officeit craigslistorg destroy), (Apple,  awesome thanks seller shipping), (Apple,  awesome yes  ), (Apple,  barcode cozy gb incluye), (Apple,  battery life hr lasted), (Apple,  birthday daughter sadness expectations), (Apple,  box headphone original came), (Apple,  brand new happywith wrap), (Apple,  brotherinlaw baught scraches scared), (Apple,  bugging purse broadband door), (Apple,  button home stick working), (Apple,  buy purchase afternoon ok), (Apple,  c verizon inspection knockoff), (Apple,  camera front solution picture), (Apple,  camera iphoneaidcom rare front), (Apple,  case protector stood defender), (Apple,  cc sinceholds longwifi unblocked), (Apple,  cell recomed powerfull ago), (Apple,  cellphone celphone practice cell), (Apple,  charge charged charging charger), (Apple,  charger bottom damage charging), (Apple,  charger cord later charging), (Apple,  charging unusedso generated port), (Apple,  christmas husband present mom), (Apple,  college student school tough), (Apple,  columbus ohio  locally), (Apple,  completing hacked goin threw), (Apple,  condition arrived packaged incomplete), (Apple,  condition ordering thank listed), (Apple,  condition thank discription perfect), (Apple,  contract phonephone renew ), (Apple,  coolest owned best samsung), (Apple,  country overseas state traveling), (Apple,  cracked recibio se wasis), (Apple,  data year planthe husband), (Apple,  daughter teenage hank father), (Apple,  de el la que), (Apple,  deal  specious fixing), (Apple,  delievery thanks everything perfect), (Apple,  delivered verizonupdated guessing obvious), (Apple,  delivery product arrived apreciated), (Apple,  delivery prompt ty fast), (Apple,  delivery quick item arrived), (Apple,  described exactly thank expediting), (Apple,  description  turbo blackunlocked), (Apple,  dialed number reciever credit), (Apple,  displayless displaying lour goodpoor), (Apple,  done job nice well), (Apple,  download prop nokiaun unavailable), (Apple,  draining specific received posting), (Apple,  droid x iphone suggested), (Apple,  dropped drop mths discounted), (Apple,  easy temperamental jump tends), (Apple,  en perfecto funciona la), (Apple,  english spanish language philippine), (Apple,  enhance seeking technological class), (Apple,  excellent excelent prompt five), (Apple,  excellent ver ad en), (Apple,  expected everything flight packed), (Apple,  expected expectation promised everithing), (Apple,  expert ideal iphone reinstall), (Apple,  facebook ram capable moneyit), (Apple,  facebook upgrade twitter beggining), (Apple,  faster slow space pretty), (Apple,  fine ok  good), (Apple,  fine perfectly work working), (Apple,  fm radio proprietary variety), (Apple,  fone bueno good muy), (Apple,  freeze freezing texter slow), (Apple,  friend bought dated paid), (Apple,  fumble birdwatching deposit opted), (Apple,  gift grandson nephew shipperthank), (Apple,  glitch like minor annoying), (Apple,  good fone oh clean), (Apple,  good vey wanted hold), (Apple,  gps map navigation ), (Apple,  grade recomended cheer fair), (Apple,  grandma came learn time), (Apple,  grandson pertaining lesson negative), (Apple,  great   ), (Apple,  great evening nonsmartphone fried), (Apple,  great friend  ), (Apple,  great lot amazing easy), (Apple,  great wonderful  ), (Apple,  greatcall delievred somehimes battery), (Apple,  gs g g iphone), (Apple,  gs info choosefor innovate), (Apple,  gs jailbreak cincinnati damaged), (Apple,  gs understandable  marvelous), (Apple,  happened month fineif quieter), (Apple,  happy alias enjoyed dougther), (Apple,  happy enjoy thankful im), ...]


In [203]:
res_neg_raw = res_neg[['Brand', 'Result']]
res_neg_new = res_neg_raw.groupby(['Brand', 'Result']).first()
res_neg_new

Empty DataFrame
Columns: []
Index: [(Apple,  advertised quickly thankyou exactly), (Apple,  bad flip modern requirement), (Apple,  bad none good ), (Apple,  battery charge hour minute), (Apple,  broken fixed broke yet), (Apple,  button fixed refused said), (Apple,  camera quality chinese front), (Apple,  charge hold would fell), (Apple,  charger cord come thing), (Apple,  crack cracked screen dropped), (Apple,  de la el que), (Apple,  described exactly mostly everything), (Apple,  dirt worn upkeep upall), (Apple,  exactly wanted needed daughter), (Apple,  excelent product bust expensive), (Apple,  excellente niceee wonderfull nicee), (Apple,  exelent ecelent expected ), (Apple,  exelente espectacular exselente esuless), (Apple,  exellent graet exclent grieta), (Apple,  expected expectation met lent), (Apple,  fast described shipped quickly), (Apple,  garbej african well nt), (Apple,  gb gb iphone gb), (Apple,  glass personal addt thus), (Apple,  glitch anyting buggy horrible), (Apple,  gracias buen producto kargia), (Apple,  gs two charger cable), (Apple,  headphone volume earphone sound), (Apple,  home button push stick), (Apple,  home key button mess), (Apple,  life battery cheap good), (Apple,  lint cleaned pulling plug), (Apple,  new brand look apple), (Apple,  new tem described wear), (Apple,  norm excelent  ), (Apple,  people better set worst), (Apple,  phone nt month one), (Apple,  price fast cheap resold), (Apple,  problem many funtionality noted), (Apple,  producto excelente recomendado recomendable), (Apple,  reception basement g low), (Apple,  refurbished multiple  opposed), (Apple,  returned defective back item), (Apple,  returning returned ordered got), (Apple,  samsung warranty  day), (Apple,  screen popped day christmas), (Apple,  shuts wo turn defective), (Apple,  slow constantly apps web), (Apple,  slow suit nostalgia purpose), (Apple,  small serf purpose practical), (Apple,  spanish language english doesnt), (Apple,  speaker blown desnot one), (Apple,  speaker sound falling disconnected), (Apple,  star whatsapp copy option), (Apple,  still count handle trying), (Apple,  terrible poor crap sucked), (Apple,  thak aaa sper thnx), (Apple,  thumb excellent  ), (Apple,  timely manner delivery time), (Apple,  unlocked sim card locked), (Apple,  upgrade update io lollipop), (Apple,  verizon activate signel get), (Apple,  water damage fixed didnt), (Apple,  work soooooooo smoth run), (Apple,  worst ever pick charge), (Nokia,  advertised quickly thankyou exactly), (Nokia,  bad flip modern requirement), (Nokia,  bad none good ), (Nokia,  battery charge hour minute), (Nokia,  bien muy si ), (Nokia,  broken fixed broke yet), (Nokia,  bueno muy bsico talkie), (Nokia,  bueno perfec buenisimo ), (Nokia,  button fixed refused said), (Nokia,  camera quality chinese front), (Nokia,  charger cord come thing), (Nokia,  crack cracked screen dropped), (Nokia,  de la el que), (Nokia,  described exactly mostly everything), (Nokia,  dirt worn upkeep upall), (Nokia,  exactly wanted needed daughter), (Nokia,  excelent excelen  ), (Nokia,  excelent excelnte excelents moment), (Nokia,  excelent product bust expensive), (Nokia,  excelente   ), (Nokia,  excelente excellente celular inversion), (Nokia,  excelente xcelente expectativas cumpli), (Nokia,  excellente niceee wonderfull nicee), (Nokia,  exelent ecelent expected ), (Nokia,  exelente espectacular exselente esuless), (Nokia,  exellent graet exclent grieta), (Nokia,  expected expectation met lent), (Nokia,  fast described shipped quickly), (Nokia,  garbej african well nt), (Nokia,  gb gb iphone gb), (Nokia,  glitch anyting buggy horrible), (Nokia,  gracias buen producto kargia), (Nokia,  gs two charger cable), (Nokia,  headphone volume earphone sound), (Nokia,  life battery cheap good), ...]

[205 rows x 0 columns]

In [191]:
res_pos_raw

,Brand,Result
0,Samsung,stoled ihad new happy
1,Samsung,phone call use nokia
2,Samsung,happy enjoy thankful im
3,Samsung,ty tryna voxox aint
4,Samsung,button home stick working
...,...,...
12155,Nokia,phone call use nokia
12156,Nokia,phone call use nokia
12157,Nokia,phone call use nokia
12158,Nokia,phone call use nokia


In [205]:
res_pos_raw.to_csv('res_pos.csv')

In [206]:
res_neg_raw.to_csv('res_neg.csv')

In [134]:
pos_topics_per_class = model_pos.topics_per_class(docs_pos, classes=df_pos['Brand Name'])
model_pos.visualize_topics_per_class(pos_topics_per_class, top_n_topics=10, normalize_frequency = True)

3it [00:00,  4.84it/s]


In [116]:
res_pos

,Topic,Count,Name,Representation,Representative_Docs,Brand Name
0,-1,1282,-1_phone_nt_one_use,"[phone, nt, one, use, like, good, work, would,...","[straight vendor phone , fact , unlocked ( use...",Samsung
1,0,161,0_s5_galaxy_s3_mini,"[s5, galaxy, s3, mini, s6, a5, samsung, imei, ...",[phone great got good price ( $ 485 ) . first ...,Samsung
2,1,117,1_camera_video_great_picture,"[camera, video, great, picture, nice, really, ...","[phone really slick . 's quite thin , feel lik...",Samsung
3,2,101,2_good_trans_far_patria,"[good, trans, far, patria, stole, god, learn, ...","[good, good, good]",Samsung
4,3,82,3_water_waterproof_dropped_proof,"[water, waterproof, dropped, proof, flashlight...",[husband plumber broken ever phone ever got wa...,Samsung
...,...,...,...,...,...,...
307,80,12,80_expensiveif_finland_transportation_fell,"[expensiveif, finland, transportation, fell, h...","[wife fell love phone , used finland , germany...",Nokia
308,81,12,81_saw_mom_dad_recently,"[saw, mom, dad, recently, wanted, bought, love...",[recently purchase mom love dad saw wanted one...,Nokia
309,82,12,82_thats_like_say_lot,"[thats, like, say, lot, plastic, little, phone...","[x x x x x x x like . thats say x x x x x x x,...",Nokia
310,83,11,83_texter_freeze_froze_numbersletters,"[texter, freeze, froze, numbersletters, tolera...","[purchased nokia c-300 back november , 2011. f...",Nokia


In [24]:
model.visualize_barchart(top_n_topics = 5, n_words = 10)

In [25]:
topics_per_class = model.topics_per_class(docs,
                                          classes = df['Brand Name'])

NameError: ignored

In [ ]:
model.visualize_topics_per_class(topics_per_class,
                                 top_n_topics = 10,
                                 normalize_frequency = True)

In [ ]:
model.visualize_topics()